* Spark MLlib Decision tree example

A college dataset (cllege.txt) is used as the input data set.

Problem statement: Try to classify colleges as Private or Public based on the features.

Data dictionary is as below:

School - Name of the college

Private - Has Yes or No indicating if the college is private or public university

Apps - Number of applications received

Accept - Number of applications accepted

Enroll - Number of new students enrolled

Top10perc - Pct. new students from top 10% of H.S. class

Top25perc - Pct. new students from top 25% of H.S. class

F_Undergrad - Number of fulltime undergraduates

P_Undergrad - Number of parttime undergraduates

Outstate - Out-of-state tuition

Room_Board -  Room and board costs

Books - Estimated book costs

Personal - Estimated personal spending

PhD - Percentage of faculty with Ph.D.’s

Terminal - Percentage of faculty with terminal degree

S_F_Ratio - Student vs Faculty ratio

Perc_alumni - Percentage of alumni who donate

Expend - Instructional expenditure per student

Grad_Rate - Graduation rate


In [1]:
#Tree methods Example
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Decisiontreeexample').getOrCreate()

In [2]:
# Load training data which should be in Hadoop at the location /user/root
data = spark.read.csv('college.csv',inferSchema=True,header=True)

In [3]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [4]:
data.head()

Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)

Transforming input dataframe into the format required for Spark MLlib algorithm

In [5]:
# Dataframe needs to be in the form of two columns
# ("label","features") where features is a vector

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [6]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [7]:
assembler = VectorAssembler(
  inputCols=['Apps',
             'Accept',
             'Enroll',
             'Top10perc',
             'Top25perc',
             'F_Undergrad',
             'P_Undergrad',
             'Outstate',
             'Room_Board',
             'Books',
             'Personal',
             'PhD',
             'Terminal',
             'S_F_Ratio',
             'perc_alumni',
             'Expend',
             'Grad_Rate'],
              outputCol="features")

In [8]:
output = assembler.transform(data)

In [9]:
type(output)

pyspark.sql.dataframe.DataFrame

In [10]:
output.show(truncate=False)

+---------------------------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+----------------------------------------------------------------------------------------------------------+
|School                                 |Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|features                                                                                                  |
+---------------------------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+----------------------------------------------------------------------------------------------------------+
|Abilene Christian University           |Yes    |1660|1232  |721   |23       |5

Data type of Private column is string with "yes" or "no" as the values.

In its place numeric values of 0 or 1 needs to be used. A column PrivateIndex is added for this purpose.

In [12]:
from pyspark.ml.feature import StringIndexer

In [13]:
indexer = StringIndexer(inputCol="Private", outputCol="PrivateIndex")
output_fixed = indexer.fit(output).transform(output)

In [14]:
output_fixed.show(5)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+------------+
|              School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|            features|PrivateIndex|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+------------+
|Abilene Christian...|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|         12|  7041|       60|[1660.0,1232.0,72...|         0.0|
|  Adelphi University|    Yes|2186|  1924|   512|       16|       29|       2683|       1227|   12280|      6450|  750|    1500| 29|      30

In [15]:
final_data = output_fixed.select("features",'PrivateIndex')

In [16]:
final_data.show()

+--------------------+------------+
|            features|PrivateIndex|
+--------------------+------------+
|[1660.0,1232.0,72...|         0.0|
|[2186.0,1924.0,51...|         0.0|
|[1428.0,1097.0,33...|         0.0|
|[417.0,349.0,137....|         0.0|
|[193.0,146.0,55.0...|         0.0|
|[587.0,479.0,158....|         0.0|
|[353.0,340.0,103....|         0.0|
|[1899.0,1720.0,48...|         0.0|
|[1038.0,839.0,227...|         0.0|
|[582.0,498.0,172....|         0.0|
|[1732.0,1425.0,47...|         0.0|
|[2652.0,1900.0,48...|         0.0|
|[1179.0,780.0,290...|         0.0|
|[1267.0,1080.0,38...|         0.0|
|[494.0,313.0,157....|         0.0|
|[1420.0,1093.0,22...|         0.0|
|[4302.0,992.0,418...|         0.0|
|[1216.0,908.0,423...|         0.0|
|[1130.0,704.0,322...|         0.0|
|[3540.0,2001.0,10...|         1.0|
+--------------------+------------+
only showing top 20 rows



In [17]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [18]:
final_data.show(10,False)

+----------------------------------------------------------------------------------------------------------+------------+
|features                                                                                                  |PrivateIndex|
+----------------------------------------------------------------------------------------------------------+------------+
|[1660.0,1232.0,721.0,23.0,52.0,2885.0,537.0,7440.0,3300.0,450.0,2200.0,70.0,78.0,18.1,12.0,7041.0,60.0]   |0.0         |
|[2186.0,1924.0,512.0,16.0,29.0,2683.0,1227.0,12280.0,6450.0,750.0,1500.0,29.0,30.0,12.2,16.0,10527.0,56.0]|0.0         |
|[1428.0,1097.0,336.0,22.0,50.0,1036.0,99.0,11250.0,3750.0,400.0,1165.0,53.0,66.0,12.9,30.0,8735.0,54.0]   |0.0         |
|[417.0,349.0,137.0,60.0,89.0,510.0,63.0,12960.0,5450.0,450.0,875.0,92.0,97.0,7.7,37.0,19016.0,59.0]       |0.0         |
|[193.0,146.0,55.0,16.0,44.0,249.0,869.0,7560.0,4120.0,800.0,1500.0,76.0,72.0,11.9,2.0,10922.0,15.0]       |0.0         |
|[587.0,479.0,158.0,38.0

DecisionTreeClassifier

In [19]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline

Create the model using default options and specifying impurity as 'entropy'.
Another option for impurity 'gini' can be also used.

In [20]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol='features',impurity='entropy')

Train the model

In [21]:
dtc_model = dtc.fit(train_data)

Test the model with test part of the input data.
Output dataframe contains:

predictionCol "prediction" - Double which is the Predicted label	

rawPredictionCol "rawPrediction" - Vector which is a Vector of length # classes (length 2 in this example), with the counts of training instance labels at the tree node which makes the prediction

probabilityCol "probability" - Vector which is a Vector of length # classes equal to rawPrediction (length 2 in this example), normalized to a multinomial distribution

In [22]:
dtc_predictions = dtc_model.transform(test_data)

In [23]:
test_data.show(5,False)

+----------------------------------------------------------------------------------------------------+------------+
|features                                                                                            |PrivateIndex|
+----------------------------------------------------------------------------------------------------+------------+
|[150.0,130.0,88.0,23.0,50.0,341.0,768.0,10300.0,4130.0,500.0,1700.0,44.0,58.0,10.2,37.0,9678.0,75.0]|0.0         |
|[244.0,198.0,82.0,12.0,33.0,1134.0,336.0,11700.0,5300.0,550.0,805.0,71.0,88.0,27.8,18.0,8694.0,58.0]|0.0         |
|[257.0,183.0,109.0,19.0,41.0,396.0,38.0,7850.0,3410.0,400.0,1500.0,55.0,70.0,10.0,15.0,7233.0,53.0] |0.0         |
|[261.0,192.0,111.0,15.0,36.0,453.0,266.0,9690.0,4300.0,500.0,500.0,57.0,77.0,9.7,35.0,9337.0,71.0]  |0.0         |
|[279.0,276.0,126.0,17.0,37.0,600.0,435.0,6844.0,2980.0,500.0,500.0,45.0,55.0,11.7,38.0,5073.0,44.0] |0.0         |
+-----------------------------------------------------------------------

In [26]:
dtc_predictions.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



Evaluation Metrics

In [24]:
dtc_predictions.show(5)

+--------------------+------------+-------------+--------------------+----------+
|            features|PrivateIndex|rawPrediction|         probability|prediction|
+--------------------+------------+-------------+--------------------+----------+
|[150.0,130.0,88.0...|         0.0|  [283.0,0.0]|           [1.0,0.0]|       0.0|
|[244.0,198.0,82.0...|         0.0|  [283.0,0.0]|           [1.0,0.0]|       0.0|
|[257.0,183.0,109....|         0.0|  [283.0,0.0]|           [1.0,0.0]|       0.0|
|[261.0,192.0,111....|         0.0|  [283.0,0.0]|           [1.0,0.0]|       0.0|
|[279.0,276.0,126....|         0.0|   [10.0,3.0]|[0.76923076923076...|       0.0|
+--------------------+------------+-------------+--------------------+----------+
only showing top 5 rows



In [27]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [28]:
# Select (prediction, true label) and compute test accuracy
acc_evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndex", predictionCol="prediction", metricName="accuracy")

In [29]:
dtc_acc = acc_evaluator.evaluate(dtc_predictions)

In [30]:
print(dtc_acc)

0.9041095890410958


In [31]:
# Display the value as a percentage with a better formatting
print("Here are the results!")
print('-'*80)
print('A single decision tree had an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('-'*80)

Here are the results!
--------------------------------------------------------------------------------
A single decision tree had an accuracy of: 90.41%
--------------------------------------------------------------------------------


In [32]:
print(dtc_model.toDebugString)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4b2ba76146a1e5391cf6) of depth 5 with 43 nodes
  If (feature 5 <= 3092.0)
   If (feature 7 <= 7245.0)
    If (feature 6 <= 201.5)
     If (feature 3 <= 15.5)
      If (feature 11 <= 64.5)
       Predict: 0.0
      Else (feature 11 > 64.5)
       Predict: 1.0
     Else (feature 3 > 15.5)
      Predict: 0.0
    Else (feature 6 > 201.5)
     If (feature 0 <= 839.0)
      If (feature 15 <= 4952.5)
       Predict: 1.0
      Else (feature 15 > 4952.5)
       Predict: 0.0
     Else (feature 0 > 839.0)
      Predict: 1.0
   Else (feature 7 > 7245.0)
    If (feature 15 <= 6738.0)
     If (feature 11 <= 70.5)
      Predict: 0.0
     Else (feature 11 > 70.5)
      If (feature 0 <= 839.0)
       Predict: 0.0
      Else (feature 0 > 839.0)
       Predict: 1.0
    Else (feature 15 > 6738.0)
     Predict: 0.0
  Else (feature 5 > 3092.0)
   If (feature 7 <= 9879.0)
    If (feature 14 <= 33.5)
     If (feature 12 <= 64.5)
      If (feature 5 <